In [1]:
import pandas as pd
import numpy as np
import re
import nltk 
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm, tqdm_notebook
import warnings
warnings.simplefilter(action='ignore')
pd.set_option('display.max_rows', 1000)

In [2]:
#read the csv
df = pd.read_csv('C:\\Users\\ighos\\Downloads\\ESV Bible\\ESV.csv', sep = '/n/n')
df = pd.DataFrame(df)
df = df.astype('string')

In [3]:
#create Book, Verse, Chapter columns, removing pointers from Text
df['Book'] = ''
df['Verse'] = ''
df['Chapter'] = ''
Book = ''
Verse = ''
for i in tqdm(range(len(df))):
        if bool(re.search(r'\s\d*\:\d*\s', df.Text[i])): 
            df.Book[i]= df.Text[i].split()[0] 
            df.Verse[i]= df.Text[i].split()[1]
            Book = df.Text[i].split()[0]
            Verse = df.Text[i].split()[1]
            df.Text[i] = df.Text[i][len(Book+' '+Verse):]
            df.Chapter[i] = Verse.split(':')[0]
            df.Verse[i] = Verse.split(':')[1] 
        else: 
            df.Book[i] = Book
            df.Verse[i] = Verse
            df.Chapter[i] = Verse.split(':')[0]
            df.Verse[i] = Verse.split(':')[1] 
            

100%|██████████| 52194/52194 [00:08<00:00, 6128.09it/s]


In [4]:
#glue all the verse's parts together in one line
j = 0
for i in tqdm(range(len(df)-1)):
    if df.Verse[i] == df.Verse[i+1] and df.Chapter[i] == df.Chapter[i+1] and df.Book[i] == df.Book[i+1]:
        if j == 0:
            df.Text[i] = df.Text[i] + ' ' + df.Text[i+1]
            j += 1
        if j == 1:
            df.Text[i-1] = df.Text[i-1] + ' ' + df.Text[i+1]
            j = 0

100%|██████████| 52193/52193 [00:01<00:00, 35259.78it/s]


In [5]:
#mark all excessive lines
df['Delete'] = ''
for i in tqdm(range(len(df)-1)):
    if df.Verse[i] == df.Verse[i+1] and df.Chapter[i] == df.Chapter[i+1] and df.Book[i] == df.Book[i+1]:
        df.Delete[i+1] = 'X'

100%|██████████| 52193/52193 [00:01<00:00, 34363.78it/s]


In [6]:
#delete non useful lines and columns
df = df.drop(df[df['Delete'] == 'X'].index)
df = df.drop(columns=['Delete'])
df = df.reset_index(drop = True)


In [7]:
#download libraries for lemmatizing and pos tagging
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('popular')
#WordNetCorpusReader.ADJ/ADJ_SAT/ADV/NOUN/VERB (which have the values "a", "s", "r", "n", "v" respectively

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ighos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ighos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\ighos\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\ighos\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\ighos\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\ighos\AppData\Roaming\nltk_dat

True

In [8]:
#tokenize all the text
df['Token'] = ''
for i in tqdm(range(len(df))):
    df.Token[i] = nltk.word_tokenize(df.Text[i])

100%|██████████| 31068/31068 [00:04<00:00, 6567.15it/s]


In [9]:
#assing for each row different token, drop 'Text' column
ds = df.explode('Token')
ds = ds.drop(columns = ['Text'])
ds = ds.reset_index(drop = True)

In [10]:
#create a column for lemmatized tokens, make all tokens upper, remove empty tokens
ds['Token_Lem'] = ''
ds.Token = ds.Token.str.upper()
ds['Token'].replace('', np.nan, inplace=True)
ds.dropna(subset=['Token'], inplace=True)
ds = ds.reset_index(drop=True)

In [11]:
#fix all the typos, drop rows with garbage tokens
ds.Book[ds.Book == '{Jo'] = '1Jo'
ds.Book[ds.Book == '2kKi'] = '2Ki'
ds.Book[ds.Book == '28a'] = '2Sa'
ds = ds[ds.Token != '2TI1:1']
ds = ds[ds.Token != '™']
ds.Token[ds.Token == '1AM'] = 'AM'
ds.Token[ds.Token == 'L'] = 'I'
ds.Token[ds.Token == '0'] = 'O'
ds = ds[ds.Token != 'E']
ds = ds[ds.Token != 'D']
ds = ds[ds.Token != 'D']
ds = ds[ds.Token != '1CO']
ds = ds[ds.Token != '2CO']
ds.Token[ds.Token == 'CAP-PADOCIA'] = 'CAPPADOCIA'
ds.Token[ds.Token == 'ZA-DOK'] = 'ZADOK'
ds.Token[ds.Token == 'RE-HOBOAM'] = 'REHOBOAM'
ds.Token[ds.Token == 'AZA-RIAH'] = 'AZARIAH'
ds.Token[ds.Token == 'AMO-RITE'] = 'AMORITE'
ds.Token[ds.Token == 'AHA-ZIAH'] = 'AHAZIAH'
ds.Token[ds.Token == 'RIGHTEOUSNESS'] = 'RIGHTEOUSNESS'
ds.Token = ds.Token[ds.Token.str.contains('\d*\:\d*') == False]

In [12]:
#find tokens 'glued' to each other, separate them to different columns
def split_and_update(row):
    if isinstance(row, str):
        match = re.match(r'\D*\,(\d*)\,(\d*)', row)
        if match:
            parts = match[0].split(',')
            matched_string = parts[0]
            return matched_string, match.group(1)+match.group(2)
        else:
            return row, None
    else:
        return row, None


# Create a new DataFrame with split values
split_values = ds['Token'].apply(lambda x:split_and_update(x))
split_df = pd.DataFrame()
tqdm_notebook().pandas()
split_values = pd.DataFrame(split_values)
split_df[['B1','B2']] = split_values['Token'].progress_apply(pd.Series)
# Update the original DataFrame
ds = pd.concat([ds, split_df], axis=1)

0it [00:00, ?it/s]

  0%|          | 0/949375 [00:00<?, ?it/s]

In [13]:
#let's see what we got
ds[ds.Token.str.contains('\D*\,\d*\,\d*') == True]

,Book,Verse,Chapter,Token,Token_Lem,B1,B2
118425,Num,50,3,"MONEY,1,365",,MONEY,1365
230496,Jdg,2,20,"GOD,400,000",,GOD,400000
274496,2Sa,6,10,"ZOBAH,20,000",,ZOBAH,20000
274513,2Sa,6,10,"TOB,12,000",,TOB,12000
297835,1Ki,14,5,"LEBANON,10,000",,LEBANON,10000
348032,2Ki,14,24,"VALOR,10,000",,VALOR,10000
348104,2Ki,16,24,"VALOR,7,0",,VALOR,70
348112,2Ki,16,24,"WORKERS,1,0",,WORKERS,10
353148,1Ch,18,5,"WAR,44,760",,WAR,44760
353246,1Ch,21,5,"CAMELS,250,000",,CAMELS,250000


In [14]:
#create additional rows and fill them with extracted tokens
new_data = []
for index, row in tqdm(ds.iterrows()):
    if pd.notna(ds.B2[index]):
        # Append the current row
        row['Token'] = row['B1']
        new_data.append(row)
        # Insert a new row with column2 and column3 swapped
        new_row = row.copy()
        new_row['Token'] = row['B2']
        new_row['B2'] = None
        new_data.append(new_row)
    if pd.isna(ds.B2[index]):
        new_data.append(row)

# Create a new DataFrame from the modified data
new_df = pd.DataFrame(new_data)

# Reset the index of the new DataFrame
new_df.reset_index(drop=True, inplace=True)
ds = new_df.drop(columns = ['B1', 'B2'])
ds = ds.reset_index(drop = True)

949375it [00:25, 36709.53it/s]


In [15]:
#check tokens with one symbol only for garbage
ds.Token[ds.Token.str.len() < 2].unique()

array([',', '.', '”', ';', 'I', 'A', "'", '?', '!', '(', ')', '’', 'O',
       '‘', '-', '1', '“', '#', '[', ']', '', '2'], dtype=object)

In [16]:
#fill column 'pos' with tags
ds.Token = ds.Token.fillna('')
ds.Token = ds.Token.astype('string').str.lower()
tqdm_notebook().pandas()
ds['pos'] = ds.Token.progress_apply(lambda x:nltk.pos_tag([x])[0][1])


0it [00:00, ?it/s]

  0%|          | 0/949445 [00:00<?, ?it/s]

In [17]:
ds['pos_lem'] = 'n'
ds.pos_lem[ds.pos.isin(['VB', 'VBD', 'VBN'])] = 'v'
ds.pos_lem[ds.pos.isin(['JJ', 'JJR'])] = 'a'
ds.pos_lem[ds.pos.isin(['JJS'])] = 's'
ds.pos_lem[ds.pos.isin(['RB', 'RBS', 'WRB'])] = 'r'

In [18]:
# ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'

In [19]:
lemmatizer = WordNetLemmatizer()
ds.Token = ds.Token.astype('string')
ds.Token = ds.Token.fillna('')
ds.Token = ds.Token.str.lower()
for i in tqdm(range(len(ds))):
    ds.Token_Lem[i] = lemmatizer.lemmatize(ds.Token[i], pos = ds.pos_lem[i])

100%|██████████| 949445/949445 [20:34<00:00, 769.10it/s] 


In [22]:
ds.Token = ds.Token.str.upper()
ds.Token_Lem = ds.Token_Lem.str.upper()

In [23]:
ds.head(100)

,Book,Verse,Chapter,Token,Token_Lem,pos,pos_lem
0,Gen,1,1,IN,IN,IN,n
1,Gen,1,1,THE,THE,DT,n
2,Gen,1,1,BEGINNING,BEGINNING,VBG,n
3,Gen,1,1,",",",",",",n
4,Gen,1,1,GOD,GOD,NN,n
5,Gen,1,1,CREATED,CREATE,VBN,v
6,Gen,1,1,THE,THE,DT,n
7,Gen,1,1,HEAVENS,HEAVEN,NNS,n
8,Gen,1,1,AND,AND,CC,n
9,Gen,1,1,THE,THE,DT,n


In [24]:
ds.pos[(ds.Token_Lem == 'RIGHTEOUSNESS') & (ds.pos == 'JJ')] = 'NN'

In [25]:
names = ['ABED-NEGO', 'ABEL-BETH', 'ABEL-BETH-MAACAH', 'ABEL-MAIM', 'ABEL-MEHOLAH', 'ABEL-MIZRAIM', 'ABEL-SHITTIM', 'ABI-ALBON', 'ABI-JAH', 'ABIM-ELECH', 'ADAMI-NEKEB', 'ADONI-BEZEK', 'ADONI-JAH', 'ADONI-ZEDEK', 'AHAZIAH', 'AHASU-ERUS', 'AHIM-ELECH', 'AHIMA-AZ', 'AHIS-AMACH', 'AHISH-AHAR', 'ALLON-BACUTH', 'ALMON-DIBLATHAIM', 'AMO-RITE', 'ANAI-AH', 'ARAM-MAACAH', 'ARAM-NAHARAIM', 'ARAM-ZOBAH', 'ARMOR-BEARER', 'ASHTEROTH-KARNAIM', 'ATAROTH-ADDAR', 'ATROTH-BETH-JOAB', 'ATROTH-SHOPHAN', 'AZARIAH', 'CAPPADOCIA', 'LEB-KAMAI', 'LEBO-HAMATH', 'LO-DEBAR', 'LYE-ABARIM', 'RAB-MAG', 'RAB-SARIS', 'RAMATH-LEHI', 'RAMATH-MIZPEH', 'RAMATHAIM-ZOPHIM', 'RAMOTH-GILEAD', 'RE-HABIAH', 'REHOBOAM', 'REGEM-MELECH', 'REHOBOTH-IR', 'RIMMON-PEREZ', 'ROMAMTI-EZER', 'UZZEN-SHEERAH', 'ZADOK', 'ZAPHENATH-PANEAH', 'ZEPHA-NIAH', 'ZERETH-SHAHAR', 'ZIM-MAH', 'ZOBAH-HAMATH']

In [26]:
ds.pos[ds.Token_Lem.isin(names)] = 'NNP'

In [27]:
numbers = ['EIGHTY-FIVE', 'EIGHTY-FOUR', 'FIFTY-FIVE', 'FORTY-FIVE', 'NINETY-NINE', 'SEVENTY-FIVE', 'SIXTY-FIVE']

In [28]:
ds.pos[ds.Token_Lem.isin(numbers)] = 'CD'
ds.pos[ds.Token_Lem == 'O'] = 'UH'

In [29]:
ds.Token_Lem[ds.Token_Lem.str.len() < 2].unique()

array([',', '.', 'U', '”', ';', 'I', 'A', "'", '?', 'L', '!', '', '(',
       ')', '’', 'O', '‘', '-', '1', '“', '#', '[', ']', '2'],
      dtype=object)

In [30]:
punct = [',', '.', '”', ';', "'", '?', '!', '(', ')', '’', '‘', '-', '“', '#', '[', ']', '...', '..', ', ', '--', '``', 'pos	CNT-Lem-Spcd
'"']

In [31]:
ds.pos[ds.Token_Lem.isin(punct)] = 'PUNCT'

In [32]:
ds.Token_Lem = ds.Token_Lem[ds.Token_Lem.str.len() > 1].str.strip('-\'.')

In [33]:
ds.pos[ds.Token_Lem == 'TO'] = 'IN'

In [34]:
pers_pron = ['I', 'YOU', 'HE', 'SHE', 'IT', 'WE', 'THEY']
pos_pron = ['MY', 'YOUR', 'HIS', 'HER', 'ITS', 'OUR', 'THEIR', 'MINE', 'YOURS', 'HIS', 'HERS', 'ITS', 'OURS', 'THEIRS']
ds.pos[ds.Token_Lem.isin(pers_pron)] = 'PRP'
ds.pos[ds.Token_Lem.isin(pos_pron)] = 'PRP$'

In [35]:
cc = ds.Token_Lem[ds.pos == 'CC'].unique()
cd = ds.Token_Lem[ds.pos == 'CD'].unique()
dt = ds.Token_Lem[ds.pos == 'DT'].unique()
ex = ds.Token_Lem[ds.pos == 'EX'].unique()
i_n = ds.Token_Lem[ds.pos == 'IN'].unique()
jjr = ds.Token_Lem[ds.pos == 'JJR'].unique()
jjs = ds.Token_Lem[ds.pos == 'JJS'].unique()
md = ds.Token_Lem[ds.pos == 'MD'].unique()
prp = ds.Token_Lem[ds.pos == 'PRP'].unique()
prps = ds.Token_Lem[ds.pos == 'PRP$'].unique()
rb = ['SO', 'TOGETHER', 'VERY', 'THUS', 'YET', 'ALONE', 'AGAIN',
       'WELL', 'LONGER', 'AWAY', 'BACK', 'ANYMORE', 'EVER', 'DOWN',
       'ABOUT', 'RIGHT', 'ALTOGETHER', 'JUST', 'ONCE', 'ASIDE', 'ELSE',
       'APART', 'SOON', 'LONG', 'EVEN', 'LATER', 'LIKEWISE', 'MEANWHILE',
       'TWICE', 'NEVERTHELESS', 'RATHER', 'ALREADY', 'LATE', 'ALMOST',
       'APIECE', 'SOMETIMES', 'ALIKE', 'NEWLY', 'AGO', 'OFTEN',
       'OTHERWISE', 'EARLIER', 'ELSEWHERE', 'SOMETIME', 'ALOFT', 'NEARBY',
       'SOMEWHAT']
rbs = ds.Token_Lem[ds.pos == 'RBS'].unique()
uh = ds.Token_Lem[ds.pos == 'UH'].unique()
vb = ds.Token_Lem[ds.pos == 'VB'].unique()
wdt = ds.Token_Lem[ds.pos == 'WDT'].unique()
wp = ds.Token_Lem[ds.pos == 'WP'].unique()
wrb = ds.Token_Lem[ds.pos == 'WRB'].unique()
ds.Token_Lem[ds.Token_Lem == 'HAD'] = 'HAVE'
ds.pos[ds.Token_Lem == 'HAVE'] = 'VB'
ds.Token_Lem[ds.Token_Lem == 'MADE'] = 'MAKE'
ds.Token_Lem[ds.Token_Lem == 'LED'] = 'LEAD'
ds.pos[ds.pos == 'VBN'] = 'VB'
ds.pos[ds.Token_Lem == 'LOST'] = 'JJ'
ds.pos[ds.Token_Lem == 'O'] = 'UH'
ds.pos[ds.Token_Lem == 'IN'] = 'IN'
ds.pos[ds.Token_Lem == 'IS'] = 'VB'
ds.pos[ds.Token_Lem == 'ARE'] = 'VB'

In [36]:
ds.pos[ds.Token_Lem.isin(cc)] = 'CC'
ds.pos[ds.Token_Lem.isin(cd)] = 'CD'
ds.pos[ds.Token_Lem.isin(dt)] = 'DT'
ds.pos[ds.Token_Lem.isin(ex)] = 'EX'
ds.pos[ds.Token_Lem.isin(i_n)] = 'IN'
ds.pos[ds.Token_Lem.isin(jjr)] = 'JJR'
ds.pos[ds.Token_Lem.isin(jjs)] = 'JJS'
ds.pos[ds.Token_Lem.isin(md)] = 'MD'
ds.pos[ds.Token_Lem.isin(prp)] = 'PRP'
ds.pos[ds.Token_Lem.isin(prps)] = 'PRPS'
ds.pos[ds.Token_Lem.isin(rb)] = 'RB'
ds.pos[ds.Token_Lem.isin(rbs)] = 'RBS'
ds.pos[ds.Token_Lem.isin(uh)] = 'UH'
ds.pos[ds.Token_Lem.isin(vb)] = 'VB'
ds.pos[ds.Token_Lem.isin(wdt)] = 'WDT'
ds.pos[ds.Token_Lem.isin(wp)] = 'WP'
ds.pos[ds.Token_Lem.isin(wrb)] = 'WRB'

In [37]:
ds.Token_Lem = ds.Token_Lem.fillna('')

In [38]:
ds.head(100)

,Book,Verse,Chapter,Token,Token_Lem,pos,pos_lem
0,Gen,1,1,IN,IN,IN,n
1,Gen,1,1,THE,THE,DT,n
2,Gen,1,1,BEGINNING,BEGINNING,VBG,n
3,Gen,1,1,",",,UH,n
4,Gen,1,1,GOD,GOD,NN,n
5,Gen,1,1,CREATED,CREATE,VB,v
6,Gen,1,1,THE,THE,DT,n
7,Gen,1,1,HEAVENS,HEAVEN,NNS,n
8,Gen,1,1,AND,AND,CC,n
9,Gen,1,1,THE,THE,DT,n


In [40]:
ds.to_csv('C:\\Users\\ighos\\Downloads\\ESV Bible\\ds.csv')
df.to_csv('C:\\Users\\ighos\\Downloads\\ESV Bible\\df.csv')

In [45]:
df = df.drop(columns = ['Token'])

In [46]:
df.head()

,Text,Book,Verse,Chapter
0,"In the beginning, God created the heavens and...",Gen,1,1
1,"The earth was without form and void, and dark...",Gen,2,1
2,"And God said,""Let there be light"","" and there...",Gen,3,1
3,And God saw that the light was good. And God ...,Gen,4,1
4,"God called the light Day, and the darkness he...",Gen,5,1


In [65]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(df.Text[0])


{'neg': 0.0, 'neu': 0.628, 'pos': 0.372, 'compound': 0.7845}

In [66]:
for i in tqdm(range(len(df))):
    verse = sia.polarity_scores(df.Text[i])
    df['neg'][i] = verse.get('neg')
    df['neu'][i] = verse.get('neu')
    df['pos'][i] = verse.get('pos')
    df['compound'][i] = verse.get('compound')

100%|██████████| 31068/31068 [00:09<00:00, 3367.75it/s]


In [74]:
df[df.compound < 0]

,Text,Book,Verse,Chapter,neg,neu,pos,compound
35,When no bush of the field was yet in the land...,Gen,5,2,0.136,0.821,0.043,-0.5423
50,The man gave names to all livestock and to th...,Gen,20,2,0.145,0.804,0.051,-0.5218
54,Therefore a man shall leave his father and hi...,Gen,24,2,0.057,0.943,0.000,-0.0516
59,"But the serpent said to the woman,""You will n...",Gen,4,3,0.290,0.513,0.197,-0.2498
65,"And he said,""I heard the sound of you in the ...",Gen,10,3,0.069,0.931,0.000,-0.1027
...,...,...,...,...,...,...,...,...
31041,"And I saw no temple in the city, for its temp...",Rev,22,21,0.091,0.823,0.086,-0.0258
31044,and its gates will never be shut by day- and ...,Rev,25,21,0.128,0.872,0.000,-0.2960
31051,And night will be no more. They will need no ...,Rev,5,22,0.131,0.806,0.063,-0.3182
31057,"Let the evildoer still do evil, and the filth...",Rev,11,22,0.279,0.721,0.000,-0.8591


In [75]:
df.to_csv('C:\\Users\\ighos\\Downloads\\ESV Bible\\df.csv')

In [80]:
ds['neg'] = ''
ds['neu'] = ''
ds['pos'] = ''
ds['compound'] = ''
for i in tqdm(range(len(ds))):
    verse = sia.polarity_scores(ds.Token_Lem[i])
    ds['neg'][i] = verse.get('neg')
    ds['neu'][i] = verse.get('neu')
    ds['pos'][i] = verse.get('pos')
    ds['compound'][i] = verse.get('compound')

100%|██████████| 949445/949445 [02:16<00:00, 6938.45it/s]


In [81]:
ds.head()

,Book,Verse,Chapter,Token,Token_Lem,pos,pos_lem,neg,neu,compound
0,Gen,1,1,IN,IN,0.0,n,0.0,1.0,0.0
1,Gen,1,1,THE,THE,0.0,n,0.0,1.0,0.0
2,Gen,1,1,BEGINNING,BEGINNING,0.0,n,0.0,1.0,0.0
3,Gen,1,1,",",,0.0,n,0.0,0.0,0.0
4,Gen,1,1,GOD,GOD,1.0,n,0.0,0.0,0.2732


In [ ]:
ds.to_csv('C:\\Users\\ighos\\Downloads\\ESV Bible\\ds.csv')